In [1]:
#Loading Libraries
import numpy as np 
import pandas as pd
import numpy as np
import os
import imutils
import dlib 
import cv2 
import imageio
from imutils import face_utils

In [2]:
people = ['F01','F02','F04','F05','F06','F07','F08','F09', 'F10','F11','M01','M02','M04','M07','M08']
data_types = ['words']
folder_enum = ['01','02','03','04','05','06','07','08','09','10']
instances = ['01','02','03','04','05','06','07','08','09','10']

words = ['Begin', 'Choose', 'Connection', 'Navigation', 'Next', 'Previous', 'Start', 'Stop', 'Hello', 'Web']          
words_di = {i:words[i] for i in range(len(words))}

In [3]:
os.listdir('D:/Lipreading/concatenated_images(5x5)')

['F01',
 'F02',
 'F04',
 'F05',
 'F06',
 'F07',
 'F08',
 'F09',
 'F10',
 'F11',
 'M01',
 'M02',
 'M04',
 'M07',
 'M08']

In [4]:
from skimage.transform import resize
import time

In [5]:
import os
import time
import imageio
import numpy as np
from skimage.transform import resize

# Define constants
MAX_WIDTH = 100
MAX_HEIGHT = 100
#max_seq_length = 40  # Updated to match the desired sequence length

t1 = time.time()

UNSEEN_VALIDATION_SPLIT = ['F07', 'M02']
UNSEEN_TEST_SPLIT = ['F04', 'M01']

# people1 = ['F01', 'F02', 'F04', 'F05', 'F06', 'F07', 'F08', 'F09', 'F10', 'F11', 'M01', 'M02', 'M04', 'M07', 'M08']
# data_types1 = ['words']
# folder_enum1 = ['01', '02', '03', '04', '05']

X_train, X_val, X_test = [], [], []
y_train, y_val, y_test = [], [], []

directory = "D:\Lipreading\concatenated_images(5x5)"

for person_id in people:
    tx1 = time.time()
    for data_type in data_types:
        for word_index, word in enumerate(folder_enum):
            print(f"Word : '{words[word_index]}'")
            path = os.path.join(directory, person_id, data_type, word)
            filelist = sorted(os.listdir(path + '/'))
            sequence = []
            for img_name in filelist:
                image = imageio.imread(path + '/' + img_name)
                image = resize(image, (MAX_WIDTH, MAX_HEIGHT))
                image = 255 * image
                # Convert to integer data type pixels.
                image = image.astype(np.uint8)
                sequence.append(image)
            pad_array = [np.zeros((MAX_WIDTH, MAX_HEIGHT))] #* (max_seq_length - len(sequence))
            sequence.extend(pad_array)  # * (max_seq_length - len(sequence)))
            # sequence = np.array(sequence)

            if person_id in UNSEEN_TEST_SPLIT:
                X_test.append(sequence)
                y_test.append(word_index)
            elif person_id in UNSEEN_VALIDATION_SPLIT:
                X_val.append(sequence)
                y_val.append(word_index)
            else:
                X_train.append(sequence)
                y_train.append(word_index)
    tx2 = time.time()
    print(f'Finished reading images for person {person_id}. Time taken : {tx2 - tx1} secs.')

t2 = time.time()
print(f"Time taken for creating constant size 3D Tensors from those concatenated lip images : {t2 - t1} secs.")


Word : 'Begin'


C:\Users\satvi\AppData\Local\Temp\ipykernel_23984\1597153363.py:35: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(path + '/' + img_name)


Word : 'Choose'
Word : 'Connection'
Word : 'Navigation'
Word : 'Next'
Word : 'Previous'
Word : 'Start'
Word : 'Stop'
Word : 'Hello'
Word : 'Web'
Finished reading images for person F01. Time taken : 1.0354979038238525 secs.
Word : 'Begin'
Word : 'Choose'
Word : 'Connection'
Word : 'Navigation'
Word : 'Next'
Word : 'Previous'
Word : 'Start'
Word : 'Stop'
Word : 'Hello'
Word : 'Web'
Finished reading images for person F02. Time taken : 0.38032007217407227 secs.
Word : 'Begin'
Word : 'Choose'
Word : 'Connection'
Word : 'Navigation'
Word : 'Next'
Word : 'Previous'
Word : 'Start'
Word : 'Stop'
Word : 'Hello'
Word : 'Web'
Finished reading images for person F04. Time taken : 0.4163553714752197 secs.
Word : 'Begin'
Word : 'Choose'
Word : 'Connection'
Word : 'Navigation'
Word : 'Next'
Word : 'Previous'
Word : 'Start'
Word : 'Stop'
Word : 'Hello'
Word : 'Web'
Finished reading images for person F05. Time taken : 0.3383033275604248 secs.
Word : 'Begin'
Word : 'Choose'
Word : 'Connection'
Word : 'Nav

In [6]:
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)

In [7]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(110, 11, 100, 100)
(20, 11, 100, 100)
(20, 11, 100, 100)


In [8]:
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

In [9]:
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(110,)
(20,)
(20,)


In [10]:
def normalize_it(X):
    v_min = X.min(axis=(2, 3), keepdims=True)
    v_max = X.max(axis=(2, 3), keepdims=True)

    # Calculate the range and replace zero values with epsilon to avoid division by zero
    range_values = np.where((v_max - v_min) == 0, np.finfo(float).eps, v_max - v_min)

    # Normalize the data
    X_normalized = np.where(range_values == 0, 0, (X - v_min) / range_values)

    return X_normalized

In [11]:
from keras.utils import to_categorical


In [12]:
X_train = normalize_it(X_train)
X_val = normalize_it(X_val)
X_test = normalize_it(X_test)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
y_val = to_categorical(y_val, 10)

from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=0)
X_test, y_test = shuffle(X_test, y_test, random_state=0)
X_val, y_val = shuffle(X_val, y_val, random_state=0)

In [13]:
X_train = np.expand_dims(X_train, axis=4)
X_val = np.expand_dims(X_val, axis=4)
X_test = np.expand_dims(X_test, axis=4)

In [14]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(110, 11, 100, 100, 1)
(20, 11, 100, 100, 1)
(20, 11, 100, 100, 1)


In [15]:
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(110, 10)
(20, 10)
(20, 10)


## RESNET

In [16]:
import cv2
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, Activation, ZeroPadding3D, TimeDistributed, LSTM, GRU, Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense, Dropout, Add, Activation
from tensorflow.keras.models import Model


In [17]:
# Define the Residual Block
def residual_block(input_tensor, filters, strides=(1, 1, 1)):
    shortcut = input_tensor

    # First convolutional layer
    x = Conv3D(filters, kernel_size=(3, 3, 3), strides=strides, padding='same')(input_tensor)
    x = Activation('relu')(x)

    # Second convolutional layer
    x = Conv3D(filters, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same')(x)

    # If dimensions change, adjust shortcut
    if strides != (1, 1, 1) or shortcut.shape[-1] != filters:
        shortcut = Conv3D(filters, kernel_size=(1, 1, 1), strides=strides, padding='valid')(shortcut)

    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

# Define the ResNet model
def create_resnet(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Conv3D(64, (7, 7, 7), strides=2, activation='relu', padding='same')(inputs)
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Residual blocks
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=128, strides=(2, 2, 2))
    x = residual_block(x, filters=128)
    x = residual_block(x, filters=256, strides=(2, 2, 2))
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=512, strides=(2, 2, 2))
    x = residual_block(x, filters=512)

    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(2048, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Create the ResNet model
resnet_model = create_resnet(input_shape=(11, 100, 100, 1), num_classes=10)

# Compile the model
resnet_model.compile(loss='categorical_crossentropy', optimizer='Adagrad', metrics=['accuracy'])

# Print model summary
resnet_model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 11, 100, 100, 1)   │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d (Conv3D)               │ (None, 6, 50, 50, 64)     │          22,016 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling3d (MaxPooling3D)  │ (None, 3, 25, 25, 64)     │               0 │ conv3d[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_1 (Conv3D)             │ (None, 3, 25, 25, 64)     │         110,656 │ max_pooling3d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation (Activation)       │ (None, 3, 25, 25, 64)     │               0 │ conv3d_1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_2 (Conv3D)             │ (None, 3, 25, 25, 64)     │         110,656 │ activation[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add (Add)                     │ (None, 3, 25, 25, 64)     │               0 │ conv3d_2[0][0],            │
│                               │                           │                 │ max_pooling3d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_1 (Activation)     │ (None, 3, 25, 25, 64)     │               0 │ add[0][0]                  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_3 (Conv3D)             │ (None, 3, 25, 25, 64)     │         110,656 │ activation_1[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_2 (Activation)     │ (None, 3, 25, 25, 64)     │               0 │ conv3d_3[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_4 (Conv3D)             │ (None, 3, 25, 25, 64)     │         110,656 │ activation_2[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_1 (Add)                   │ (None, 3, 25, 25, 64)     │               0 │ conv3d_4[0][0],            │
│                               │                           │                 │ activation_1[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_3 (Activation)     │ (None, 3, 25, 25, 64)     │               0 │ add_1[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_5 (Conv3D)             │ (None, 2, 13, 13, 128)    │         221,312 │ activation_3[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_4 (Activation)     │ (None, 2, 13, 13, 128)    │               0 │ conv3d_5[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_6 (Conv3D)             │ (None, 2, 13, 13, 128)    │         442,496 │ activation_4[0][0]         │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 77,212,810 (294.54 MB)

 Trainable params: 77,212,810 (294.54 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
t1 = time.time()
history = resnet_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)
t2 = time.time()
print()
print(f"Training time : {t2 - t1} secs.")

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 18s 4s/step - accuracy: 0.1290 - loss: 2.3033 - val_accuracy: 0.1000 - val_loss: 2.3025
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.0749 - loss: 2.3056 - val_accuracy: 0.1000 - val_loss: 2.3022
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.1431 - loss: 2.2979 - val_accuracy: 0.1000 - val_loss: 2.3020
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.0369 - loss: 2.3085 - val_accuracy: 0.1000 - val_loss: 2.3021
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.1129 - loss: 2.3008 - val_accuracy: 0.1000 - val_loss: 2.3020
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.1108 - loss: 2.3047 - val_accuracy: 0.1000 - val_loss: 2.3020
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.0838 - loss: 2.3049 - val_accuracy: 0.1000 - val_loss: 2.3020
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.1311 - loss: 2.3001 - val_accuracy: 0.1000 - val_loss: 2.3021
Epoch 9/

## Inception-V3

In [19]:
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, concatenate, Flatten, Dense, Dropout
from tensorflow.keras.models import Model

# Define the Inception module
def inception_module(x, filters):
    branch1x1 = Conv3D(filters=filters[0], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)

    branch3x3 = Conv3D(filters=filters[1], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)
    branch3x3 = Conv3D(filters=filters[2], kernel_size=(3, 3, 3), padding='same', activation='relu')(branch3x3)

    branch5x5 = Conv3D(filters=filters[3], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)
    branch5x5 = Conv3D(filters=filters[4], kernel_size=(5, 5, 5), padding='same', activation='relu')(branch5x5)

    branch_pool = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 1, 1), padding='same')(x)
    branch_pool = Conv3D(filters=filters[5], kernel_size=(1, 1, 1), padding='same', activation='relu')(branch_pool)

    output = concatenate([branch1x1, branch3x3, branch5x5, branch_pool], axis=-1)
    return output

# Define the Inception-V3 model
def create_inception_v3(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # Initial convolutional layer
    x = Conv3D(64, kernel_size=(7, 7, 7), strides=(2, 2, 2), padding='same', activation='relu')(inputs)
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=(2, 2, 2), padding='same')(x)

    # Inception blocks
    x = inception_module(x, filters=[64, 96, 128, 16, 32, 32])
    x = inception_module(x, filters=[128, 128, 192, 32, 96, 64])
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=(2, 2, 2), padding='same')(x)

    # Flatten and fully connected layers
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)

    # Output layer
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Create the Inception-V3 model
inception_v3_model = create_inception_v3(input_shape=(11, 100, 100, 1), num_classes=10)

# Compile the model
inception_v3_model.compile(loss='categorical_crossentropy', optimizer='Adagrad', metrics=['accuracy'])

# Print model summary
inception_v3_model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 11, 100, 100, 1)   │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_20 (Conv3D)            │ (None, 6, 50, 50, 64)     │          22,016 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling3d_1               │ (None, 3, 25, 25, 64)     │               0 │ conv3d_20[0][0]            │
│ (MaxPooling3D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_22 (Conv3D)            │ (None, 3, 25, 25, 96)     │           6,240 │ max_pooling3d_1[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_24 (Conv3D)            │ (None, 3, 25, 25, 16)     │           1,040 │ max_pooling3d_1[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling3d_2               │ (None, 3, 25, 25, 64)     │               0 │ max_pooling3d_1[0][0]      │
│ (MaxPooling3D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_21 (Conv3D)            │ (None, 3, 25, 25, 64)     │           4,160 │ max_pooling3d_1[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_23 (Conv3D)            │ (None, 3, 25, 25, 128)    │         331,904 │ conv3d_22[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_25 (Conv3D)            │ (None, 3, 25, 25, 32)     │          64,032 │ conv3d_24[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_26 (Conv3D)            │ (None, 3, 25, 25, 32)     │           2,080 │ max_pooling3d_2[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 3, 25, 25, 256)    │               0 │ conv3d_21[0][0],           │
│                               │                           │                 │ conv3d_23[0][0],           │
│                               │                           │                 │ conv3d_25[0][0],           │
│                               │                           │                 │ conv3d_26[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_28 (Conv3D)            │ (None, 3, 25, 25, 128)    │          32,896 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_30 (Conv3D)            │ (None, 3, 25, 25, 32)     │           8,224 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling3d_3               │ (None, 3, 25, 25, 256)    │               0 │ concatenate[0][0]          │
│ (MaxPooling3D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 168,234,490 (641.76 MB)

 Trainable params: 168,234,490 (641.76 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
t1 = time.time()
history = inception_v3_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)
t2 = time.time()
print()
print(f"Training time : {t2 - t1} secs.")

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 53s 9s/step - accuracy: 0.0833 - loss: 2.2976 - val_accuracy: 0.1000 - val_loss: 2.3075
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 5s/step - accuracy: 0.0546 - loss: 2.3273 - val_accuracy: 0.1000 - val_loss: 2.3071
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 21s 5s/step - accuracy: 0.1311 - loss: 2.3297 - val_accuracy: 0.1000 - val_loss: 2.3056
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 18s 5s/step - accuracy: 0.1004 - loss: 2.3002 - val_accuracy: 0.1000 - val_loss: 2.3043
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 5s/step - accuracy: 0.1103 - loss: 2.3154 - val_accuracy: 0.1000 - val_loss: 2.3044
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 20s 5s/step - accuracy: 0.1790 - loss: 2.2865 - val_accuracy: 0.1000 - val_loss: 2.3040
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 19s 5s/step - accuracy: 0.0900 - loss: 2.3082 - val_accuracy: 0.1000 - val_loss: 2.3044
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 24s 6s/step - accuracy: 0.1155 - loss: 2.3103 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 9/

## VGG-16

In [29]:
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model

# Define the VGG16 model
def create_vgg16(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    
    # Block 1
    x = Conv3D(64, kernel_size=(3, 3, 3), padding='same', activation='relu')(inputs)
    x = Conv3D(64, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 2
    x = Conv3D(128, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(128, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 3
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 4
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 5
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Flatten and fully connected layers
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    # Output layer
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Create the VGG16 model
vgg16_model = create_vgg16(input_shape=(11, 100, 100, 1), num_classes=10)

# Compile the model
vgg16_model.compile(loss='categorical_crossentropy', optimizer='Adagrad', metrics=['accuracy'])

# Print model summary
vgg16_model.summary()


Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)           │ (None, 11, 100, 100, 1)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_66 (Conv3D)                   │ (None, 11, 100, 100, 64)    │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_67 (Conv3D)                   │ (None, 11, 100, 100, 64)    │         110,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_10 (MaxPooling3D)      │ (None, 6, 50, 50, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_68 (Conv3D)                   │ (None, 6, 50, 50, 128)      │         221,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_69 (Conv3D)                   │ (None, 6, 50, 50, 128)      │         442,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_11 (MaxPooling3D)      │ (None, 3, 25, 25, 128)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_70 (Conv3D)                   │ (None, 3, 25, 25, 256)      │         884,992 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_71 (Conv3D)                   │ (None, 3, 25, 25, 256)      │       1,769,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_72 (Conv3D)                   │ (None, 3, 25, 25, 256)      │       1,769,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_12 (MaxPooling3D)      │ (None, 2, 13, 13, 256)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_73 (Conv3D)                   │ (None, 2, 13, 13, 512)      │       3,539,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_74 (Conv3D)                   │ (None, 2, 13, 13, 512)      │       7,078,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_75 (Conv3D)                   │ (None, 2, 13, 13, 512)      │       7,078,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_13 (MaxPooling3D)      │ (None, 1, 7, 7, 512)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_76 (Conv3D)                   │ (None, 1, 7, 7, 512)        │       7,078,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_77 (Conv3D)                   │ (None, 1, 7, 7, 512)        │       7,078,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_78 (Conv3D)                   │ (None, 1, 7, 7, 512)        │       7,078,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_14 (MaxPooling3D)      │ (None, 1, 4, 4, 512)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 4096)                │      33,558,528 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 94,512,970 (360.54 MB)

 Trainable params: 94,512,970 (360.54 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
t1 = time.time()
history = vgg16_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)
t2 = time.time()
print()
print(f"Training time : {t2 - t1} secs.")

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 212s 53s/step - accuracy: 0.0807 - loss: 2.3026 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 2/10


## ResNet+SVM

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist # Example dataset
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense, Dropout, Activation, Add
from tensorflow.keras.models import Model
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# # Load example dataset (Replace this with your actual dataset)
# (X_train, y_train), (X_test, y_test) = mnist.load_data()
# X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
# X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
# y_train = np.eye(10)[y_train]
# y_test = np.eye(10)[y_test]

# Define the Residual Block
def residual_block(input_tensor, filters, strides=(1, 1, 1)):
    shortcut = input_tensor

    # First convolutional layer
    x = Conv3D(filters, kernel_size=(3, 3, 3), strides=strides, padding='same')(input_tensor)
    x = Activation('relu')(x)

    # Second convolutional layer
    x = Conv3D(filters, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same')(x)

    # If dimensions change, adjust shortcut
    if strides != (1, 1, 1) or shortcut.shape[-1] != filters:
        shortcut = Conv3D(filters, kernel_size=(1, 1, 1), strides=strides, padding='valid')(shortcut)

    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

# Define the ResNet model without the last layer
def create_resnet_base(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv3D(64, (7, 7, 7), strides=2, activation='relu', padding='same')(inputs)
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Residual blocks
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=128, strides=(2, 2, 2))
    x = residual_block(x, filters=128)
    x = residual_block(x, filters=256, strides=(2, 2, 2))
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=512, strides=(2, 2, 2))
    x = residual_block(x, filters=512)

    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the ResNet base model
resnet_base_model = create_resnet_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = resnet_base_model.predict(X_train)
X_test_features = resnet_base_model.predict(X_test)

# Train SVM classifier
svm_classifier = SVC(kernel='linear')
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_classifier", svm_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# # ROC curve
# y_score = pipeline.decision_function(X_test_features)
# fpr, tpr, _ = roc_curve(np.argmax(y_test, axis=1), y_score)
# roc_auc = auc(fpr, tpr)

# plt.figure()
# plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic')
# plt.legend(loc="lower right")
# plt.show()


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 483ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step
Accuracy: 0.2
Precision: 0.15666666666666668
Recall: 0.2
F1 Score: 0.1488095238095238


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Inception-V3+SVM

In [22]:
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, concatenate, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Define the Inception module
def inception_module(x, filters):
    branch1x1 = Conv3D(filters=filters[0], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)

    branch3x3 = Conv3D(filters=filters[1], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)
    branch3x3 = Conv3D(filters=filters[2], kernel_size=(3, 3, 3), padding='same', activation='relu')(branch3x3)

    branch5x5 = Conv3D(filters=filters[3], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)
    branch5x5 = Conv3D(filters=filters[4], kernel_size=(5, 5, 5), padding='same', activation='relu')(branch5x5)

    branch_pool = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 1, 1), padding='same')(x)
    branch_pool = Conv3D(filters=filters[5], kernel_size=(1, 1, 1), padding='same', activation='relu')(branch_pool)

    output = concatenate([branch1x1, branch3x3, branch5x5, branch_pool], axis=-1)
    return output

# Define the Inception-V3 model without the last layer
def create_inception_v3_base(input_shape):
    inputs = Input(shape=input_shape)

    # Initial convolutional layer
    x = Conv3D(64, kernel_size=(7, 7, 7), strides=(2, 2, 2), padding='same', activation='relu')(inputs)
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Inception blocks
    x = inception_module(x, filters=[64, 96, 128, 16, 32, 32])
    x = inception_module(x, filters=[128, 128, 192, 32, 96, 64])
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Flatten and fully connected layers
    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the Inception-V3 base model
inception_v3_base_model = create_inception_v3_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = inception_v3_base_model.predict(X_train)
X_test_features = inception_v3_base_model.predict(X_test)

# Train SVM classifier
svm_classifier = SVC(kernel='linear')
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_classifier", svm_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 624ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 434ms/step
Accuracy: 0.5
Precision: 0.55
Recall: 0.5
F1 Score: 0.4833333333333333


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## VGG16+SVM

In [16]:
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Define the VGG-16 model without the last layer
def create_vgg16_base(input_shape):
    inputs = Input(shape=input_shape)
    
    # Block 1
    x = Conv3D(64, kernel_size=(3, 3, 3), padding='same', activation='relu')(inputs)
    x = Conv3D(64, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 2
    x = Conv3D(128, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(128, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 3
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 4
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 5
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Flatten and fully connected layers
    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the VGG-16 base model
vgg16_base_model = create_vgg16_base(input_shape=(11, 100, 100, 1))

# Assuming X_train and X_test are already defined and preprocessed
# Extract features
X_train_features = vgg16_base_model.predict(X_train)
X_test_features = vgg16_base_model.predict(X_test)

# Train SVM classifier
svm_classifier = SVC(kernel='linear')
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_classifier", svm_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 21s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Accuracy: 0.15
Precision: 0.11333333333333333
Recall: 0.15
F1 Score: 0.09019607843137253


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Define the VGG-16 model with Batch Normalization and Dropout
def create_vgg16_base(input_shape):
    inputs = Input(shape=input_shape)
    
    # Block 1
    x = Conv3D(64, kernel_size=(3, 3, 3), padding='same', activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Conv3D(64, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 2
    x = Conv3D(128, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv3D(128, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 3
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 4
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 5
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Flatten and fully connected layers
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the VGG-16 base model
vgg16_base_model = create_vgg16_base(input_shape=(11, 100, 100, 1))

# Assuming X_train and X_test are already defined and preprocessed
# Extract features
X_train_features = vgg16_base_model.predict(X_train)
X_test_features = vgg16_base_model.predict(X_test)

# Train SVM classifier
svm_classifier = SVC(kernel='linear')
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_classifier", svm_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Accuracy: 0.25
Precision: 0.22261904761904758
Recall: 0.25
F1 Score: 0.21222222222222223


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Resnet+KNN

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist # Example dataset
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense, Dropout, Activation, Add
from tensorflow.keras.models import Model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# # Load example dataset (Replace this with your actual dataset)
# (X_train, y_train), (X_test, y_test) = mnist.load_data()
# X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
# X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
# y_train = np.eye(10)[y_train]
# y_test = np.eye(10)[y_test]

# Define the Residual Block
def residual_block(input_tensor, filters, strides=(1, 1, 1)):
    shortcut = input_tensor

    # First convolutional layer
    x = Conv3D(filters, kernel_size=(3, 3, 3), strides=strides, padding='same')(input_tensor)
    x = Activation('relu')(x)

    # Second convolutional layer
    x = Conv3D(filters, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same')(x)

    # If dimensions change, adjust shortcut
    if strides != (1, 1, 1) or shortcut.shape[-1] != filters:
        shortcut = Conv3D(filters, kernel_size=(1, 1, 1), strides=strides, padding='valid')(shortcut)

    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

# Define the ResNet model without the last layer
def create_resnet_base(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv3D(64, (7, 7, 7), strides=2, activation='relu', padding='same')(inputs)
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Residual blocks
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=128, strides=(2, 2, 2))
    x = residual_block(x, filters=128)
    x = residual_block(x, filters=256, strides=(2, 2, 2))
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=512, strides=(2, 2, 2))
    x = residual_block(x, filters=512)

    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the ResNet base model
resnet_base_model = create_resnet_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = resnet_base_model.predict(X_train)
X_test_features = resnet_base_model.predict(X_test)

# Train KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("knn_classifier", knn_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# # ROC curve
# y_score = pipeline.decision_function(X_test_features)
# fpr, tpr, _ = roc_curve(np.argmax(y_test, axis=1), y_score)
# roc_auc = auc(fpr, tpr)

# plt.figure()
# plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic')
# plt.legend(loc="lower right")
# plt.show()


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 401ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
Accuracy: 0.15
Precision: 0.04583333333333333
Recall: 0.15
F1 Score: 0.06222222222222222


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Inception-V3+KNN

In [18]:
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, concatenate, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Define the Inception module
def inception_module(x, filters):
    branch1x1 = Conv3D(filters=filters[0], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)

    branch3x3 = Conv3D(filters=filters[1], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)
    branch3x3 = Conv3D(filters=filters[2], kernel_size=(3, 3, 3), padding='same', activation='relu')(branch3x3)

    branch5x5 = Conv3D(filters=filters[3], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)
    branch5x5 = Conv3D(filters=filters[4], kernel_size=(5, 5, 5), padding='same', activation='relu')(branch5x5)

    branch_pool = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 1, 1), padding='same')(x)
    branch_pool = Conv3D(filters=filters[5], kernel_size=(1, 1, 1), padding='same', activation='relu')(branch_pool)

    output = concatenate([branch1x1, branch3x3, branch5x5, branch_pool], axis=-1)
    return output

# Define the Inception-V3 model without the last layer
def create_inception_v3_base(input_shape):
    inputs = Input(shape=input_shape)

    # Initial convolutional layer
    x = Conv3D(64, kernel_size=(7, 7, 7), strides=(2, 2, 2), padding='same', activation='relu')(inputs)
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Inception blocks
    x = inception_module(x, filters=[64, 96, 128, 16, 32, 32])
    x = inception_module(x, filters=[128, 128, 192, 32, 96, 64])
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Flatten and fully connected layers
    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the Inception-V3 base model
inception_v3_base_model = create_inception_v3_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = inception_v3_base_model.predict(X_train)
X_test_features = inception_v3_base_model.predict(X_test)

# Train KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("knn_classifier", knn_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 463ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step
Accuracy: 0.15
Precision: 0.08095238095238096
Recall: 0.15
F1 Score: 0.09722222222222222


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## VGG-16+KNN

In [22]:
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Define the VGG-16 model without the last layer
def create_vgg16_base(input_shape):
    inputs = Input(shape=input_shape)
    
    # Block 1
    x = Conv3D(64, kernel_size=(3, 3, 3), padding='same', activation='relu')(inputs)
    x = Conv3D(64, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 2
    x = Conv3D(128, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(128, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 3
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 4
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 5
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Flatten and fully connected layers
    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the VGG-16 base model
vgg16_base_model = create_vgg16_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = vgg16_base_model.predict(X_train)
X_test_features = vgg16_base_model.predict(X_test)

# Train KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=17)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("knn_classifier", knn_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 17s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Accuracy: 0.15
Precision: 0.05
Recall: 0.15
F1 Score: 0.06857142857142857


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Resnet+Ada Boost

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist # Example dataset
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense, Dropout, Activation, Add
from tensorflow.keras.models import Model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# # Load example dataset (Replace this with your actual dataset)
# (X_train, y_train), (X_test, y_test) = mnist.load_data()
# X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
# X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
# y_train = np.eye(10)[y_train]
# y_test = np.eye(10)[y_test]

# Define the Residual Block
def residual_block(input_tensor, filters, strides=(1, 1, 1)):
    shortcut = input_tensor

    # First convolutional layer
    x = Conv3D(filters, kernel_size=(3, 3, 3), strides=strides, padding='same')(input_tensor)
    x = Activation('relu')(x)

    # Second convolutional layer
    x = Conv3D(filters, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same')(x)

    # If dimensions change, adjust shortcut
    if strides != (1, 1, 1) or shortcut.shape[-1] != filters:
        shortcut = Conv3D(filters, kernel_size=(1, 1, 1), strides=strides, padding='valid')(shortcut)

    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

# Define the ResNet model without the last layer
def create_resnet_base(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv3D(64, (7, 7, 7), strides=2, activation='relu', padding='same')(inputs)
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Residual blocks
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=128, strides=(2, 2, 2))
    x = residual_block(x, filters=128)
    x = residual_block(x, filters=256, strides=(2, 2, 2))
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=512, strides=(2, 2, 2))
    x = residual_block(x, filters=512)

    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the ResNet base model
resnet_base_model = create_resnet_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = resnet_base_model.predict(X_train)
X_test_features = resnet_base_model.predict(X_test)

# Train AdaBoost classifier
adaboost_classifier = AdaBoostClassifier(n_estimators=80, learning_rate=1.0)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("adaboost_classifier", adaboost_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 334ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step
Accuracy: 0.15
Precision: 0.04509803921568627
Recall: 0.15
F1 Score: 0.06105263157894737


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist # Example dataset
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense, Dropout, Activation, Add
from tensorflow.keras.models import Model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# # Load example dataset (Replace this with your actual dataset)
# (X_train, y_train), (X_test, y_test) = mnist.load_data()
# X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
# X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
# y_train = np.eye(10)[y_train]
# y_test = np.eye(10)[y_test]

# Define the Residual Block
def residual_block(input_tensor, filters, strides=(1, 1, 1)):
    shortcut = input_tensor

    # First convolutional layer
    x = Conv3D(filters, kernel_size=(3, 3, 3), strides=strides, padding='same')(input_tensor)
    x = Activation('relu')(x)

    # Second convolutional layer
    x = Conv3D(filters, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same')(x)

    # If dimensions change, adjust shortcut
    if strides != (1, 1, 1) or shortcut.shape[-1] != filters:
        shortcut = Conv3D(filters, kernel_size=(1, 1, 1), strides=strides, padding='valid')(shortcut)

    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

# Define the ResNet model without the last layer
def create_resnet_base(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv3D(64, (7, 7, 7), strides=2, activation='relu', padding='same')(inputs)
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Residual blocks
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=128, strides=(2, 2, 2))
    x = residual_block(x, filters=128)
    x = residual_block(x, filters=256, strides=(2, 2, 2))
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=512, strides=(2, 2, 2))
    x = residual_block(x, filters=512)

    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the ResNet base model
resnet_base_model = create_resnet_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = resnet_base_model.predict(X_train)
X_test_features = resnet_base_model.predict(X_test)

# Train AdaBoost classifier
adaboost_classifier = AdaBoostClassifier(n_estimators=100, learning_rate=0.4)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("adaboost_classifier", adaboost_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 334ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step
Accuracy: 0.3
Precision: 0.30333333333333334
Recall: 0.3
F1 Score: 0.2852380952380952


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist # Example dataset
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense, Dropout, Activation, Add
from tensorflow.keras.models import Model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# # Load example dataset (Replace this with your actual dataset)
# (X_train, y_train), (X_test, y_test) = mnist.load_data()
# X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
# X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
# y_train = np.eye(10)[y_train]
# y_test = np.eye(10)[y_test]

# Define the Residual Block
def residual_block(input_tensor, filters, strides=(1, 1, 1)):
    shortcut = input_tensor

    # First convolutional layer
    x = Conv3D(filters, kernel_size=(3, 3, 3), strides=strides, padding='same')(input_tensor)
    x = Activation('relu')(x)

    # Second convolutional layer
    x = Conv3D(filters, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same')(x)

    # If dimensions change, adjust shortcut
    if strides != (1, 1, 1) or shortcut.shape[-1] != filters:
        shortcut = Conv3D(filters, kernel_size=(1, 1, 1), strides=strides, padding='valid')(shortcut)

    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

# Define the ResNet model without the last layer
def create_resnet_base(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv3D(64, (7, 7, 7), strides=2, activation='relu', padding='same')(inputs)
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Residual blocks
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=128, strides=(2, 2, 2))
    x = residual_block(x, filters=128)
    x = residual_block(x, filters=256, strides=(2, 2, 2))
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=512, strides=(2, 2, 2))
    x = residual_block(x, filters=512)

    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the ResNet base model
resnet_base_model = create_resnet_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = resnet_base_model.predict(X_train)
X_test_features = resnet_base_model.predict(X_test)

# Train AdaBoost classifier
adaboost_classifier = AdaBoostClassifier(n_estimators=500, learning_rate=0.05)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("adaboost_classifier", adaboost_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 328ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
Accuracy: 0.3
Precision: 0.37
Recall: 0.3
F1 Score: 0.2833333333333333


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [43]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist # Example dataset
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense, Dropout, Activation, Add
from tensorflow.keras.models import Model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# # Load example dataset (Replace this with your actual dataset)
# (X_train, y_train), (X_test, y_test) = mnist.load_data()
# X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
# X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
# y_train = np.eye(10)[y_train]
# y_test = np.eye(10)[y_test]

# Define the Residual Block
def residual_block(input_tensor, filters, strides=(1, 1, 1)):
    shortcut = input_tensor

    # First convolutional layer
    x = Conv3D(filters, kernel_size=(3, 3, 3), strides=strides, padding='same')(input_tensor)
    x = Activation('relu')(x)

    # Second convolutional layer
    x = Conv3D(filters, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same')(x)

    # If dimensions change, adjust shortcut
    if strides != (1, 1, 1) or shortcut.shape[-1] != filters:
        shortcut = Conv3D(filters, kernel_size=(1, 1, 1), strides=strides, padding='valid')(shortcut)

    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

# Define the ResNet model without the last layer
def create_resnet_base(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv3D(64, (7, 7, 7), strides=2, activation='relu', padding='same')(inputs)
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Residual blocks
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=128, strides=(2, 2, 2))
    x = residual_block(x, filters=128)
    x = residual_block(x, filters=256, strides=(2, 2, 2))
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=512, strides=(2, 2, 2))
    x = residual_block(x, filters=512)

    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the ResNet base model
resnet_base_model = create_resnet_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = resnet_base_model.predict(X_train)
X_test_features = resnet_base_model.predict(X_test)

# Train AdaBoost classifier
adaboost_classifier = AdaBoostClassifier(n_estimators=150, learning_rate=0.01)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("adaboost_classifier", adaboost_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 340ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
Accuracy: 0.35
Precision: 0.23055555555555554
Recall: 0.35
F1 Score: 0.2596969696969697


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [62]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist # Example dataset
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense, Dropout, Activation, Add
from tensorflow.keras.models import Model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# # Load example dataset (Replace this with your actual dataset)
# (X_train, y_train), (X_test, y_test) = mnist.load_data()
# X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
# X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
# y_train = np.eye(10)[y_train]
# y_test = np.eye(10)[y_test]

# Define the Residual Block
def residual_block(input_tensor, filters, strides=(1, 1, 1)):
    shortcut = input_tensor

    # First convolutional layer
    x = Conv3D(filters, kernel_size=(3, 3, 3), strides=strides, padding='same')(input_tensor)
    x = Activation('relu')(x)

    # Second convolutional layer
    x = Conv3D(filters, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same')(x)

    # If dimensions change, adjust shortcut
    if strides != (1, 1, 1) or shortcut.shape[-1] != filters:
        shortcut = Conv3D(filters, kernel_size=(1, 1, 1), strides=strides, padding='valid')(shortcut)

    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

# Define the ResNet model without the last layer
def create_resnet_base(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv3D(64, (7, 7, 7), strides=2, activation='relu', padding='same')(inputs)
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Residual blocks
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=128, strides=(2, 2, 2))
    x = residual_block(x, filters=128)
    x = residual_block(x, filters=256, strides=(2, 2, 2))
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=512, strides=(2, 2, 2))
    x = residual_block(x, filters=512)

    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the ResNet base model
resnet_base_model = create_resnet_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = resnet_base_model.predict(X_train)
X_test_features = resnet_base_model.predict(X_test)

# Train AdaBoost classifier
adaboost_classifier = AdaBoostClassifier(n_estimators=200, learning_rate=0.001)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("adaboost_classifier", adaboost_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 342ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
Accuracy: 0.2
Precision: 0.14166666666666666
Recall: 0.2
F1 Score: 0.16333333333333333


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Inception-V3+Ada Boost

In [31]:
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, concatenate, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Define the Inception module
def inception_module(x, filters):
    branch1x1 = Conv3D(filters=filters[0], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)

    branch3x3 = Conv3D(filters=filters[1], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)
    branch3x3 = Conv3D(filters=filters[2], kernel_size=(3, 3, 3), padding='same', activation='relu')(branch3x3)

    branch5x5 = Conv3D(filters=filters[3], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)
    branch5x5 = Conv3D(filters=filters[4], kernel_size=(5, 5, 5), padding='same', activation='relu')(branch5x5)

    branch_pool = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 1, 1), padding='same')(x)
    branch_pool = Conv3D(filters=filters[5], kernel_size=(1, 1, 1), padding='same', activation='relu')(branch_pool)

    output = concatenate([branch1x1, branch3x3, branch5x5, branch_pool], axis=-1)
    return output

# Define the Inception-V3 model without the last layer
def create_inception_v3_base(input_shape):
    inputs = Input(shape=input_shape)

    # Initial convolutional layer
    x = Conv3D(64, kernel_size=(7, 7, 7), strides=(2, 2, 2), padding='same', activation='relu')(inputs)
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Inception blocks
    x = inception_module(x, filters=[64, 96, 128, 16, 32, 32])
    x = inception_module(x, filters=[128, 128, 192, 32, 96, 64])
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Flatten and fully connected layers
    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the Inception-V3 base model
inception_v3_base_model = create_inception_v3_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = inception_v3_base_model.predict(X_train)
X_test_features = inception_v3_base_model.predict(X_test)

# Train AdaBoost classifier
adaboost_classifier = AdaBoostClassifier(n_estimators=80, learning_rate=0.5)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("adaboost_classifier", adaboost_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 444ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
Accuracy: 0.2
Precision: 0.10666666666666666
Recall: 0.2
F1 Score: 0.13714285714285718


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, concatenate, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Define the Inception module
def inception_module(x, filters):
    branch1x1 = Conv3D(filters=filters[0], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)

    branch3x3 = Conv3D(filters=filters[1], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)
    branch3x3 = Conv3D(filters=filters[2], kernel_size=(3, 3, 3), padding='same', activation='relu')(branch3x3)

    branch5x5 = Conv3D(filters=filters[3], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)
    branch5x5 = Conv3D(filters=filters[4], kernel_size=(5, 5, 5), padding='same', activation='relu')(branch5x5)

    branch_pool = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 1, 1), padding='same')(x)
    branch_pool = Conv3D(filters=filters[5], kernel_size=(1, 1, 1), padding='same', activation='relu')(branch_pool)

    output = concatenate([branch1x1, branch3x3, branch5x5, branch_pool], axis=-1)
    return output

# Define the Inception-V3 model without the last layer
def create_inception_v3_base(input_shape):
    inputs = Input(shape=input_shape)

    # Initial convolutional layer
    x = Conv3D(64, kernel_size=(7, 7, 7), strides=(2, 2, 2), padding='same', activation='relu')(inputs)
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Inception blocks
    x = inception_module(x, filters=[64, 96, 128, 16, 32, 32])
    x = inception_module(x, filters=[128, 128, 192, 32, 96, 64])
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Flatten and fully connected layers
    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the Inception-V3 base model
inception_v3_base_model = create_inception_v3_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = inception_v3_base_model.predict(X_train)
X_test_features = inception_v3_base_model.predict(X_test)

# Train AdaBoost classifier
adaboost_classifier = AdaBoostClassifier(n_estimators=100, learning_rate=0.4)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("adaboost_classifier", adaboost_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 446ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step
Accuracy: 0.3
Precision: 0.36538461538461536
Recall: 0.3
F1 Score: 0.27666666666666667


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, concatenate, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Define the Inception module
def inception_module(x, filters):
    branch1x1 = Conv3D(filters=filters[0], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)

    branch3x3 = Conv3D(filters=filters[1], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)
    branch3x3 = Conv3D(filters=filters[2], kernel_size=(3, 3, 3), padding='same', activation='relu')(branch3x3)

    branch5x5 = Conv3D(filters=filters[3], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)
    branch5x5 = Conv3D(filters=filters[4], kernel_size=(5, 5, 5), padding='same', activation='relu')(branch5x5)

    branch_pool = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 1, 1), padding='same')(x)
    branch_pool = Conv3D(filters=filters[5], kernel_size=(1, 1, 1), padding='same', activation='relu')(branch_pool)

    output = concatenate([branch1x1, branch3x3, branch5x5, branch_pool], axis=-1)
    return output

# Define the Inception-V3 model without the last layer
def create_inception_v3_base(input_shape):
    inputs = Input(shape=input_shape)

    # Initial convolutional layer
    x = Conv3D(64, kernel_size=(7, 7, 7), strides=(2, 2, 2), padding='same', activation='relu')(inputs)
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Inception blocks
    x = inception_module(x, filters=[64, 96, 128, 16, 32, 32])
    x = inception_module(x, filters=[128, 128, 192, 32, 96, 64])
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Flatten and fully connected layers
    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the Inception-V3 base model
inception_v3_base_model = create_inception_v3_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = inception_v3_base_model.predict(X_train)
X_test_features = inception_v3_base_model.predict(X_test)

# Train AdaBoost classifier
adaboost_classifier = AdaBoostClassifier(n_estimators=150, learning_rate=0.2)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("adaboost_classifier", adaboost_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 449ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step
Accuracy: 0.3
Precision: 0.4033333333333333
Recall: 0.3
F1 Score: 0.31857142857142856


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## VGG-16+Ada Boost

In [64]:
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten
from tensorflow.keras.models import Model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Define the VGG-16 model without the last layer
def create_vgg16_base(input_shape):
    inputs = Input(shape=input_shape)
    
    # Block 1
    x = Conv3D(64, kernel_size=(3, 3, 3), padding='same', activation='relu')(inputs)
    x = Conv3D(64, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 2
    x = Conv3D(128, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(128, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 3
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 4
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 5
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Flatten and fully connected layers
    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the VGG-16 base model
vgg16_base_model = create_vgg16_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = vgg16_base_model.predict(X_train)
X_test_features = vgg16_base_model.predict(X_test)

# Train AdaBoost classifier
adaboost_classifier = AdaBoostClassifier(n_estimators=120, learning_rate=0.5)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("adaboost_classifier", adaboost_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 16s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Accuracy: 0.2
Precision: 0.11428571428571428
Recall: 0.2
F1 Score: 0.13722222222222222


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [67]:
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten
from tensorflow.keras.models import Model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Define the VGG-16 model without the last layer
def create_vgg16_base(input_shape):
    inputs = Input(shape=input_shape)
    
    # Block 1
    x = Conv3D(64, kernel_size=(3, 3, 3), padding='same', activation='relu')(inputs)
    x = Conv3D(64, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 2
    x = Conv3D(128, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(128, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 3
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 4
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 5
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Flatten and fully connected layers
    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the VGG-16 base model
vgg16_base_model = create_vgg16_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = vgg16_base_model.predict(X_train)
X_test_features = vgg16_base_model.predict(X_test)

# Train AdaBoost classifier
adaboost_classifier = AdaBoostClassifier(n_estimators=120, learning_rate=0.005)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("adaboost_classifier", adaboost_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 16s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Accuracy: 0.25
Precision: 0.18333333333333332
Recall: 0.25
F1 Score: 0.18


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [69]:
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten
from tensorflow.keras.models import Model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Define the VGG-16 model without the last layer
def create_vgg16_base(input_shape):
    inputs = Input(shape=input_shape)
    
    # Block 1
    x = Conv3D(64, kernel_size=(3, 3, 3), padding='same', activation='relu')(inputs)
    x = Conv3D(64, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 2
    x = Conv3D(128, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(128, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 3
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 4
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 5
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Flatten and fully connected layers
    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the VGG-16 base model
vgg16_base_model = create_vgg16_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = vgg16_base_model.predict(X_train)
X_test_features = vgg16_base_model.predict(X_test)

# Train AdaBoost classifier
adaboost_classifier = AdaBoostClassifier(n_estimators=300, learning_rate=0.005)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("adaboost_classifier", adaboost_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 16s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Accuracy: 0.3
Precision: 0.15888888888888889
Recall: 0.3
F1 Score: 0.19493506493506493


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## VGG-16+Random Forest

In [70]:
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten
from tensorflow.keras.models import Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Define the VGG-16 model without the last layer
def create_vgg16_base(input_shape):
    inputs = Input(shape=input_shape)
    
    # Block 1
    x = Conv3D(64, kernel_size=(3, 3, 3), padding='same', activation='relu')(inputs)
    x = Conv3D(64, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 2
    x = Conv3D(128, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(128, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 3
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 4
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 5
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Flatten and fully connected layers
    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the VGG-16 base model
vgg16_base_model = create_vgg16_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = vgg16_base_model.predict(X_train)
X_test_features = vgg16_base_model.predict(X_test)

# Train Random Forest classifier
random_forest_classifier = RandomForestClassifier(n_estimators=300, max_depth=10, random_state=42)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("random_forest_classifier", random_forest_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 17s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Accuracy: 0.3
Precision: 0.2785714285714286
Recall: 0.3
F1 Score: 0.24444444444444446


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [72]:
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten
from tensorflow.keras.models import Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Define the VGG-16 model without the last layer
def create_vgg16_base(input_shape):
    inputs = Input(shape=input_shape)
    
    # Block 1
    x = Conv3D(64, kernel_size=(3, 3, 3), padding='same', activation='relu')(inputs)
    x = Conv3D(64, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 2
    x = Conv3D(128, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(128, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 3
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(256, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 4
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Block 5
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = Conv3D(512, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    # Flatten and fully connected layers
    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the VGG-16 base model
vgg16_base_model = create_vgg16_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = vgg16_base_model.predict(X_train)
X_test_features = vgg16_base_model.predict(X_test)

# Train Random Forest classifier
random_forest_classifier = RandomForestClassifier(n_estimators=500, max_depth=50, random_state=42)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("random_forest_classifier", random_forest_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 17s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Accuracy: 0.35
Precision: 0.32
Recall: 0.35
F1 Score: 0.2866666666666667


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Inception-V3+Random Forest

In [73]:
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, concatenate, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Define the Inception module
def inception_module(x, filters):
    branch1x1 = Conv3D(filters=filters[0], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)

    branch3x3 = Conv3D(filters=filters[1], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)
    branch3x3 = Conv3D(filters=filters[2], kernel_size=(3, 3, 3), padding='same', activation='relu')(branch3x3)

    branch5x5 = Conv3D(filters=filters[3], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)
    branch5x5 = Conv3D(filters=filters[4], kernel_size=(5, 5, 5), padding='same', activation='relu')(branch5x5)

    branch_pool = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 1, 1), padding='same')(x)
    branch_pool = Conv3D(filters=filters[5], kernel_size=(1, 1, 1), padding='same', activation='relu')(branch_pool)

    output = concatenate([branch1x1, branch3x3, branch5x5, branch_pool], axis=-1)
    return output

# Define the Inception-V3 model without the last layer
def create_inception_v3_base(input_shape):
    inputs = Input(shape=input_shape)

    # Initial convolutional layer
    x = Conv3D(64, kernel_size=(7, 7, 7), strides=(2, 2, 2), padding='same', activation='relu')(inputs)
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Inception blocks
    x = inception_module(x, filters=[64, 96, 128, 16, 32, 32])
    x = inception_module(x, filters=[128, 128, 192, 32, 96, 64])
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Flatten and fully connected layers
    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the Inception-V3 base model
inception_v3_base_model = create_inception_v3_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = inception_v3_base_model.predict(X_train)
X_test_features = inception_v3_base_model.predict(X_test)

# Train Random Forest classifier
random_forest_classifier = RandomForestClassifier(n_estimators=300, max_depth=10, random_state=42)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("random_forest_classifier", random_forest_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 571ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step
Accuracy: 0.4
Precision: 0.4333333333333333
Recall: 0.4
F1 Score: 0.37


In [75]:
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, concatenate, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Define the Inception module
def inception_module(x, filters):
    branch1x1 = Conv3D(filters=filters[0], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)

    branch3x3 = Conv3D(filters=filters[1], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)
    branch3x3 = Conv3D(filters=filters[2], kernel_size=(3, 3, 3), padding='same', activation='relu')(branch3x3)

    branch5x5 = Conv3D(filters=filters[3], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)
    branch5x5 = Conv3D(filters=filters[4], kernel_size=(5, 5, 5), padding='same', activation='relu')(branch5x5)

    branch_pool = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 1, 1), padding='same')(x)
    branch_pool = Conv3D(filters=filters[5], kernel_size=(1, 1, 1), padding='same', activation='relu')(branch_pool)

    output = concatenate([branch1x1, branch3x3, branch5x5, branch_pool], axis=-1)
    return output

# Define the Inception-V3 model without the last layer
def create_inception_v3_base(input_shape):
    inputs = Input(shape=input_shape)

    # Initial convolutional layer
    x = Conv3D(64, kernel_size=(7, 7, 7), strides=(2, 2, 2), padding='same', activation='relu')(inputs)
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Inception blocks
    x = inception_module(x, filters=[64, 96, 128, 16, 32, 32])
    x = inception_module(x, filters=[128, 128, 192, 32, 96, 64])
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Flatten and fully connected layers
    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the Inception-V3 base model
inception_v3_base_model = create_inception_v3_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = inception_v3_base_model.predict(X_train)
X_test_features = inception_v3_base_model.predict(X_test)

# Train Random Forest classifier
random_forest_classifier = RandomForestClassifier(n_estimators=350, max_depth=20, random_state=42)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("random_forest_classifier", random_forest_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 456ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 362ms/step
Accuracy: 0.3
Precision: 0.3933333333333333
Recall: 0.3
F1 Score: 0.30666666666666664


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [76]:
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, concatenate, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Define the Inception module
def inception_module(x, filters):
    branch1x1 = Conv3D(filters=filters[0], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)

    branch3x3 = Conv3D(filters=filters[1], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)
    branch3x3 = Conv3D(filters=filters[2], kernel_size=(3, 3, 3), padding='same', activation='relu')(branch3x3)

    branch5x5 = Conv3D(filters=filters[3], kernel_size=(1, 1, 1), padding='same', activation='relu')(x)
    branch5x5 = Conv3D(filters=filters[4], kernel_size=(5, 5, 5), padding='same', activation='relu')(branch5x5)

    branch_pool = MaxPooling3D(pool_size=(3, 3, 3), strides=(1, 1, 1), padding='same')(x)
    branch_pool = Conv3D(filters=filters[5], kernel_size=(1, 1, 1), padding='same', activation='relu')(branch_pool)

    output = concatenate([branch1x1, branch3x3, branch5x5, branch_pool], axis=-1)
    return output

# Define the Inception-V3 model without the last layer
def create_inception_v3_base(input_shape):
    inputs = Input(shape=input_shape)

    # Initial convolutional layer
    x = Conv3D(64, kernel_size=(7, 7, 7), strides=(2, 2, 2), padding='same', activation='relu')(inputs)
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Inception blocks
    x = inception_module(x, filters=[64, 96, 128, 16, 32, 32])
    x = inception_module(x, filters=[128, 128, 192, 32, 96, 64])
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Flatten and fully connected layers
    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the Inception-V3 base model
inception_v3_base_model = create_inception_v3_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = inception_v3_base_model.predict(X_train)
X_test_features = inception_v3_base_model.predict(X_test)

# Train Random Forest classifier
random_forest_classifier = RandomForestClassifier(n_estimators=400, max_depth=30, random_state=42)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("random_forest_classifier", random_forest_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 448ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step
Accuracy: 0.4
Precision: 0.4222222222222222
Recall: 0.4
F1 Score: 0.3696969696969697


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Resnet+Random Forest

In [77]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense, Dropout, Activation, Add
from tensorflow.keras.models import Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Define the Residual Block
def residual_block(input_tensor, filters, strides=(1, 1, 1)):
    shortcut = input_tensor

    # First convolutional layer
    x = Conv3D(filters, kernel_size=(3, 3, 3), strides=strides, padding='same')(input_tensor)
    x = Activation('relu')(x)

    # Second convolutional layer
    x = Conv3D(filters, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same')(x)

    # If dimensions change, adjust shortcut
    if strides != (1, 1, 1) or shortcut.shape[-1] != filters:
        shortcut = Conv3D(filters, kernel_size=(1, 1, 1), strides=strides, padding='valid')(shortcut)

    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

# Define the ResNet model without the last layer
def create_resnet_base(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv3D(64, (7, 7, 7), strides=2, activation='relu', padding='same')(inputs)
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Residual blocks
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=128, strides=(2, 2, 2))
    x = residual_block(x, filters=128)
    x = residual_block(x, filters=256, strides=(2, 2, 2))
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=512, strides=(2, 2, 2))
    x = residual_block(x, filters=512)

    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the ResNet base model
resnet_base_model = create_resnet_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = resnet_base_model.predict(X_train)
X_test_features = resnet_base_model.predict(X_test)

# Train Random Forest classifier
random_forest_classifier = RandomForestClassifier(n_estimators=300, max_depth=10, random_state=42)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("random_forest_classifier", random_forest_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 348ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
Accuracy: 0.6
Precision: 0.5833333333333333
Recall: 0.6
F1 Score: 0.5533333333333333


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [86]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense, Dropout, Activation, Add
from tensorflow.keras.models import Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Define the Residual Block
def residual_block(input_tensor, filters, strides=(1, 1, 1)):
    shortcut = input_tensor

    # First convolutional layer
    x = Conv3D(filters, kernel_size=(3, 3, 3), strides=strides, padding='same')(input_tensor)
    x = Activation('relu')(x)

    # Second convolutional layer
    x = Conv3D(filters, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same')(x)

    # If dimensions change, adjust shortcut
    if strides != (1, 1, 1) or shortcut.shape[-1] != filters:
        shortcut = Conv3D(filters, kernel_size=(1, 1, 1), strides=strides, padding='valid')(shortcut)

    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

# Define the ResNet model without the last layer
def create_resnet_base(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv3D(64, (7, 7, 7), strides=2, activation='relu', padding='same')(inputs)
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Residual blocks
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=128, strides=(2, 2, 2))
    x = residual_block(x, filters=128)
    x = residual_block(x, filters=256, strides=(2, 2, 2))
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=512, strides=(2, 2, 2))
    x = residual_block(x, filters=512)

    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the ResNet base model
resnet_base_model = create_resnet_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = resnet_base_model.predict(X_train)
X_test_features = resnet_base_model.predict(X_test)

# Train Random Forest classifier
random_forest_classifier = RandomForestClassifier(n_estimators=1000, max_depth=90, random_state=32)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("random_forest_classifier", random_forest_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 354ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step
Accuracy: 0.5
Precision: 0.6042857142857143
Recall: 0.5
F1 Score: 0.49603174603174605


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [88]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense, Dropout, Activation, Add
from tensorflow.keras.models import Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Define the Residual Block
def residual_block(input_tensor, filters, strides=(1, 1, 1)):
    shortcut = input_tensor

    # First convolutional layer
    x = Conv3D(filters, kernel_size=(3, 3, 3), strides=strides, padding='same')(input_tensor)
    x = Activation('relu')(x)

    # Second convolutional layer
    x = Conv3D(filters, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same')(x)

    # If dimensions change, adjust shortcut
    if strides != (1, 1, 1) or shortcut.shape[-1] != filters:
        shortcut = Conv3D(filters, kernel_size=(1, 1, 1), strides=strides, padding='valid')(shortcut)

    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

# Define the ResNet model without the last layer
def create_resnet_base(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv3D(64, (7, 7, 7), strides=2, activation='relu', padding='same')(inputs)
    x = MaxPooling3D(pool_size=(3, 3, 3), strides=2, padding='same')(x)

    # Residual blocks
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=128, strides=(2, 2, 2))
    x = residual_block(x, filters=128)
    x = residual_block(x, filters=256, strides=(2, 2, 2))
    x = residual_block(x, filters=256)
    x = residual_block(x, filters=512, strides=(2, 2, 2))
    x = residual_block(x, filters=512)

    x = Flatten()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the ResNet base model
resnet_base_model = create_resnet_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = resnet_base_model.predict(X_train)
X_test_features = resnet_base_model.predict(X_test)

# Train Random Forest classifier
random_forest_classifier = RandomForestClassifier(n_estimators=500, max_depth=100, random_state=32)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("random_forest_classifier", random_forest_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 353ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
Accuracy: 0.35
Precision: 0.3611111111111111
Recall: 0.35
F1 Score: 0.3215151515151515


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Mobile Net

In [21]:
from tensorflow.keras.layers import Input, Conv3D, BatchNormalization, ReLU, GlobalAveragePooling3D, Dense, Dropout, Add, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adagrad

def depthwise_separable_conv3d(input_tensor, pointwise_filters, strides=(1, 1, 1)):
    # Depthwise convolution (simulated using grouped Conv3D)
    channels = input_tensor.shape[-1]
    depthwise_conv = Conv3D(channels, kernel_size=(3, 3, 3), strides=strides, padding='same', groups=channels)(input_tensor)
    x = BatchNormalization()(depthwise_conv)
    x = ReLU()(x)
    
    # Pointwise convolution
    pointwise_conv = Conv3D(pointwise_filters, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='same')(x)
    x = BatchNormalization()(pointwise_conv)
    x = ReLU()(x)
    return x

def create_mobilenet(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # Initial convolution and pooling layer
    x = Conv3D(32, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # MobileNet blocks
    x = depthwise_separable_conv3d(x, 64)
    x = depthwise_separable_conv3d(x, 128, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 128)
    x = depthwise_separable_conv3d(x, 256, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 256)
    x = depthwise_separable_conv3d(x, 512, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 512)

    # Global pooling and dense layers
    x = GlobalAveragePooling3D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Create the MobileNet model
mobilenet_model = create_mobilenet(input_shape=(11, 100, 100, 1), num_classes=10)

# Compile the model
mobilenet_model.compile(loss='categorical_crossentropy', optimizer=Adagrad(), metrics=['accuracy'])

# Print model summary
mobilenet_model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 11, 100, 100, 1)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d (Conv3D)                      │ (None, 6, 50, 50, 32)       │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 6, 50, 50, 32)       │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu (ReLU)                         │ (None, 6, 50, 50, 32)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_1 (Conv3D)                    │ (None, 6, 50, 50, 32)       │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 6, 50, 50, 32)       │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_1 (ReLU)                       │ (None, 6, 50, 50, 32)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_2 (Conv3D)                    │ (None, 6, 50, 50, 64)       │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 6, 50, 50, 64)       │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_2 (ReLU)                       │ (None, 6, 50, 50, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_3 (Conv3D)                    │ (None, 3, 25, 25, 64)       │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 3, 25, 25, 64)       │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_3 (ReLU)                       │ (None, 3, 25, 25, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_4 (Conv3D)                    │ (None, 3, 25, 25, 128)      │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 3, 25, 25, 128)      │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_4 (ReLU)                       │ (None, 3, 25, 25, 128)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_5 (Conv3D)                    │ (None, 3, 25, 25, 128)      │           3,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 3, 25, 25, 128)      │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 1,756,490 (6.70 MB)

 Trainable params: 1,749,962 (6.68 MB)

 Non-trainable params: 6,528 (25.50 KB)

In [22]:
t1 = time.time()
history = mobilenet_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)
t2 = time.time()
print()
print(f"Training time : {t2 - t1} secs.")

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 113s 26s/step - accuracy: 0.1410 - loss: 2.5284 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 104s 25s/step - accuracy: 0.1228 - loss: 2.5440 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 105s 26s/step - accuracy: 0.1103 - loss: 2.4329 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 106s 26s/step - accuracy: 0.0947 - loss: 2.4341 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 105s 26s/step - accuracy: 0.1212 - loss: 2.3785 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 105s 26s/step - accuracy: 0.1400 - loss: 2.4200 - val_accuracy: 0.1000 - val_loss: 2.3027
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 107s 26s/step - accuracy: 0.0749 - loss: 2.4682 - val_accuracy: 0.1000 - val_loss: 2.3027
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 101s 25s/step - accuracy: 0.0926 - loss: 2.4777 - val_accuracy: 0.1000 - val_loss:

## Mobile Net+Random Forest

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv3D, GlobalAveragePooling3D, Dense, Dropout, Activation, BatchNormalization, ReLU
from tensorflow.keras.models import Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Define the depthwise separable convolution block
def depthwise_separable_conv3d(input_tensor, pointwise_filters, strides=(1, 1, 1)):
    channels = input_tensor.shape[-1]
    depthwise_conv = Conv3D(channels, kernel_size=(3, 3, 3), strides=strides, padding='same', groups=channels)(input_tensor)
    x = BatchNormalization()(depthwise_conv)
    x = ReLU()(x)
    
    pointwise_conv = Conv3D(pointwise_filters, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='same')(x)
    x = BatchNormalization()(pointwise_conv)
    x = ReLU()(x)
    return x

# Define the MobileNet model without the last layer
def create_mobilenet_base(input_shape):
    inputs = Input(shape=input_shape)

    # Initial convolution and pooling layer
    x = Conv3D(32, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # MobileNet blocks
    x = depthwise_separable_conv3d(x, 64)
    x = depthwise_separable_conv3d(x, 128, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 128)
    x = depthwise_separable_conv3d(x, 256, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 256)
    x = depthwise_separable_conv3d(x, 512, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 512)

    # Global pooling
    x = GlobalAveragePooling3D()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Assume X_train, X_test, y_train, y_test are predefined
# Example data initialization
# X_train, X_test, y_train, y_test = train_test_split(...)

# Create the MobileNet base model
mobilenet_base_model = create_mobilenet_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = mobilenet_base_model.predict(X_train)
X_test_features = mobilenet_base_model.predict(X_test)

# Train Random Forest classifier
random_forest_classifier = RandomForestClassifier(n_estimators=200, max_depth=30, random_state=42)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("random_forest_classifier", random_forest_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


3/4 ━━━━━━━━━━━━━━━━━━━━ 8s 9s/step WARNING:tensorflow:5 out of the last 11 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x000002199A8CA8C0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
4/4 ━━━━━━━━━━━━━━━━━━━━ 33s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
Accuracy: 0.15
Precision: 0.1392857142857143
Recall: 0.15
F1 Score: 0.1222222222222222


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv3D, GlobalAveragePooling3D, Dense, Dropout, Activation, BatchNormalization, ReLU
from tensorflow.keras.models import Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Define the depthwise separable convolution block
def depthwise_separable_conv3d(input_tensor, pointwise_filters, strides=(1, 1, 1)):
    channels = input_tensor.shape[-1]
    depthwise_conv = Conv3D(channels, kernel_size=(3, 3, 3), strides=strides, padding='same', groups=channels)(input_tensor)
    x = BatchNormalization()(depthwise_conv)
    x = ReLU()(x)
    
    pointwise_conv = Conv3D(pointwise_filters, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='same')(x)
    x = BatchNormalization()(pointwise_conv)
    x = ReLU()(x)
    return x

# Define the MobileNet model without the last layer
def create_mobilenet_base(input_shape):
    inputs = Input(shape=input_shape)

    # Initial convolution and pooling layer
    x = Conv3D(32, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # MobileNet blocks
    x = depthwise_separable_conv3d(x, 64)
    x = depthwise_separable_conv3d(x, 128, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 128)
    x = depthwise_separable_conv3d(x, 256, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 256)
    x = depthwise_separable_conv3d(x, 512, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 512)

    # Global pooling
    x = GlobalAveragePooling3D()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Assume X_train, X_test, y_train, y_test are predefined
# Example data initialization
# X_train, X_test, y_train, y_test = train_test_split(...)

# Create the MobileNet base model
mobilenet_base_model = create_mobilenet_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = mobilenet_base_model.predict(X_train)
X_test_features = mobilenet_base_model.predict(X_test)

# Train Random Forest classifier
random_forest_classifier = RandomForestClassifier(n_estimators=2000, max_depth=30, random_state=42)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("random_forest_classifier", random_forest_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 32s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
Accuracy: 0.3
Precision: 0.12833333333333333
Recall: 0.3
F1 Score: 0.17857142857142855


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv3D, GlobalAveragePooling3D, Dense, Dropout, Activation, BatchNormalization, ReLU
from tensorflow.keras.models import Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Define the depthwise separable convolution block
def depthwise_separable_conv3d(input_tensor, pointwise_filters, strides=(1, 1, 1)):
    channels = input_tensor.shape[-1]
    depthwise_conv = Conv3D(channels, kernel_size=(3, 3, 3), strides=strides, padding='same', groups=channels)(input_tensor)
    x = BatchNormalization()(depthwise_conv)
    x = ReLU()(x)
    
    pointwise_conv = Conv3D(pointwise_filters, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='same')(x)
    x = BatchNormalization()(pointwise_conv)
    x = ReLU()(x)
    return x

# Define the MobileNet model without the last layer
def create_mobilenet_base(input_shape):
    inputs = Input(shape=input_shape)

    # Initial convolution and pooling layer
    x = Conv3D(32, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # MobileNet blocks
    x = depthwise_separable_conv3d(x, 64)
    x = depthwise_separable_conv3d(x, 128, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 128)
    x = depthwise_separable_conv3d(x, 256, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 256)
    x = depthwise_separable_conv3d(x, 512, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 512)

    # Global pooling
    x = GlobalAveragePooling3D()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Assume X_train, X_test, y_train, y_test are predefined
# Example data initialization
# X_train, X_test, y_train, y_test = train_test_split(...)

# Create the MobileNet base model
mobilenet_base_model = create_mobilenet_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = mobilenet_base_model.predict(X_train)
X_test_features = mobilenet_base_model.predict(X_test)

# Train Random Forest classifier
random_forest_classifier = RandomForestClassifier(n_estimators=1500, max_depth=70, random_state=42)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("random_forest_classifier", random_forest_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 32s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
Accuracy: 0.25
Precision: 0.10833333333333332
Recall: 0.25
F1 Score: 0.14190476190476192


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv3D, GlobalAveragePooling3D, Dense, Dropout, Activation, BatchNormalization, ReLU
from tensorflow.keras.models import Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Define the depthwise separable convolution block
def depthwise_separable_conv3d(input_tensor, pointwise_filters, strides=(1, 1, 1)):
    channels = input_tensor.shape[-1]
    depthwise_conv = Conv3D(channels, kernel_size=(3, 3, 3), strides=strides, padding='same', groups=channels)(input_tensor)
    x = BatchNormalization()(depthwise_conv)
    x = ReLU()(x)
    
    pointwise_conv = Conv3D(pointwise_filters, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='same')(x)
    x = BatchNormalization()(pointwise_conv)
    x = ReLU()(x)
    return x

# Define the MobileNet model without the last layer
def create_mobilenet_base(input_shape):
    inputs = Input(shape=input_shape)

    # Initial convolution and pooling layer
    x = Conv3D(32, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # MobileNet blocks
    x = depthwise_separable_conv3d(x, 64)
    x = depthwise_separable_conv3d(x, 128, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 128)
    x = depthwise_separable_conv3d(x, 256, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 256)
    x = depthwise_separable_conv3d(x, 512, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 512)

    # Global pooling
    x = GlobalAveragePooling3D()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Assume X_train, X_test, y_train, y_test are predefined
# Example data initialization
# X_train, X_test, y_train, y_test = train_test_split(...)

# Create the MobileNet base model
mobilenet_base_model = create_mobilenet_base(input_shape=(11, 100, 100, 1))

# Extract features
X_train_features = mobilenet_base_model.predict(X_train)
X_test_features = mobilenet_base_model.predict(X_test)

# Train Random Forest classifier
random_forest_classifier = RandomForestClassifier(n_estimators=1800, max_depth=10, random_state=32)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("random_forest_classifier", random_forest_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 32s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
Accuracy: 0.2
Precision: 0.0976190476190476
Recall: 0.2
F1 Score: 0.1288888888888889


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Mobile Net+Ada Boost

In [35]:
from tensorflow.keras.layers import Input, Conv3D, GlobalAveragePooling3D, Dense, Dropout, Activation, BatchNormalization, ReLU
from tensorflow.keras.models import Model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Define the depthwise separable convolution block
def depthwise_separable_conv3d(input_tensor, pointwise_filters, strides=(1, 1, 1)):
    channels = input_tensor.shape[-1]
    depthwise_conv = Conv3D(channels, kernel_size=(3, 3, 3), strides=strides, padding='same', groups=channels)(input_tensor)
    x = BatchNormalization()(depthwise_conv)
    x = ReLU()(x)
    
    pointwise_conv = Conv3D(pointwise_filters, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='same')(x)
    x = BatchNormalization()(pointwise_conv)
    x = ReLU()(x)
    return x

# Define the MobileNet model without the last layer
def create_mobilenet_base(input_shape):
    inputs = Input(shape=input_shape)

    # Initial convolution and pooling layer
    x = Conv3D(32, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # MobileNet blocks
    x = depthwise_separable_conv3d(x, 64)
    x = depthwise_separable_conv3d(x, 128, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 128)
    x = depthwise_separable_conv3d(x, 256, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 256)
    x = depthwise_separable_conv3d(x, 512, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 512)

    # Global pooling
    x = GlobalAveragePooling3D()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the MobileNet base model
mobilenet_base_model = create_mobilenet_base(input_shape=(11, 100, 100, 1))

# Assume X_train, X_test, y_train, y_test are predefined
# Example data initialization
# X_train, X_test, y_train, y_test = train_test_split(...)

# Extract features
X_train_features = mobilenet_base_model.predict(X_train)
X_test_features = mobilenet_base_model.predict(X_test)

# Train AdaBoost classifier
adaboost_classifier = AdaBoostClassifier(n_estimators=200, learning_rate=0.001)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("adaboost_classifier", adaboost_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 33s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
Accuracy: 0.2
Precision: 0.05818181818181818
Recall: 0.2
F1 Score: 0.08791208791208792


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
from tensorflow.keras.layers import Input, Conv3D, GlobalAveragePooling3D, Dense, Dropout, Activation, BatchNormalization, ReLU
from tensorflow.keras.models import Model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Define the depthwise separable convolution block
def depthwise_separable_conv3d(input_tensor, pointwise_filters, strides=(1, 1, 1)):
    channels = input_tensor.shape[-1]
    depthwise_conv = Conv3D(channels, kernel_size=(3, 3, 3), strides=strides, padding='same', groups=channels)(input_tensor)
    x = BatchNormalization()(depthwise_conv)
    x = ReLU()(x)
    
    pointwise_conv = Conv3D(pointwise_filters, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='same')(x)
    x = BatchNormalization()(pointwise_conv)
    x = ReLU()(x)
    return x

# Define the MobileNet model without the last layer
def create_mobilenet_base(input_shape):
    inputs = Input(shape=input_shape)

    # Initial convolution and pooling layer
    x = Conv3D(32, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # MobileNet blocks
    x = depthwise_separable_conv3d(x, 64)
    x = depthwise_separable_conv3d(x, 128, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 128)
    x = depthwise_separable_conv3d(x, 256, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 256)
    x = depthwise_separable_conv3d(x, 512, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 512)

    # Global pooling
    x = GlobalAveragePooling3D()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the MobileNet base model
mobilenet_base_model = create_mobilenet_base(input_shape=(11, 100, 100, 1))

# Assume X_train, X_test, y_train, y_test are predefined
# Example data initialization
# X_train, X_test, y_train, y_test = train_test_split(...)

# Extract features
X_train_features = mobilenet_base_model.predict(X_train)
X_test_features = mobilenet_base_model.predict(X_test)

# Train AdaBoost classifier
adaboost_classifier = AdaBoostClassifier(n_estimators=2000, learning_rate=0.005)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("adaboost_classifier", adaboost_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 33s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
Accuracy: 0.15
Precision: 0.13095238095238096
Recall: 0.15
F1 Score: 0.11388888888888889


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Mobile Net+SVM

In [38]:
from tensorflow.keras.layers import Input, Conv3D, GlobalAveragePooling3D, Dense, Dropout, Activation, BatchNormalization, ReLU
from tensorflow.keras.models import Model
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Define the depthwise separable convolution block
def depthwise_separable_conv3d(input_tensor, pointwise_filters, strides=(1, 1, 1)):
    channels = input_tensor.shape[-1]
    depthwise_conv = Conv3D(channels, kernel_size=(3, 3, 3), strides=strides, padding='same', groups=channels)(input_tensor)
    x = BatchNormalization()(depthwise_conv)
    x = ReLU()(x)
    
    pointwise_conv = Conv3D(pointwise_filters, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='same')(x)
    x = BatchNormalization()(pointwise_conv)
    x = ReLU()(x)
    return x

# Define the MobileNet model without the last layer
def create_mobilenet_base(input_shape):
    inputs = Input(shape=input_shape)

    # Initial convolution and pooling layer
    x = Conv3D(32, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # MobileNet blocks
    x = depthwise_separable_conv3d(x, 64)
    x = depthwise_separable_conv3d(x, 128, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 128)
    x = depthwise_separable_conv3d(x, 256, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 256)
    x = depthwise_separable_conv3d(x, 512, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 512)

    # Global pooling
    x = GlobalAveragePooling3D()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the MobileNet base model
mobilenet_base_model = create_mobilenet_base(input_shape=(11, 100, 100, 1))

# Assume X_train, X_test, y_train, y_test are predefined
# Example data initialization
# X_train, X_test, y_train, y_test = train_test_split(...)

# Extract features
X_train_features = mobilenet_base_model.predict(X_train)
X_test_features = mobilenet_base_model.predict(X_test)

# Train SVM classifier
svm_classifier = SVC(kernel='linear', C=1.0)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_classifier", svm_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 33s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
Accuracy: 0.3
Precision: 0.24333333333333332
Recall: 0.3
F1 Score: 0.24


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
from tensorflow.keras.layers import Input, Conv3D, GlobalAveragePooling3D, Dense, Dropout, Activation, BatchNormalization, ReLU
from tensorflow.keras.models import Model
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Define the depthwise separable convolution block
def depthwise_separable_conv3d(input_tensor, pointwise_filters, strides=(1, 1, 1)):
    channels = input_tensor.shape[-1]
    depthwise_conv = Conv3D(channels, kernel_size=(3, 3, 3), strides=strides, padding='same', groups=channels)(input_tensor)
    x = BatchNormalization()(depthwise_conv)
    x = ReLU()(x)
    
    pointwise_conv = Conv3D(pointwise_filters, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='same')(x)
    x = BatchNormalization()(pointwise_conv)
    x = ReLU()(x)
    return x

# Define the MobileNet model without the last layer
def create_mobilenet_base(input_shape):
    inputs = Input(shape=input_shape)

    # Initial convolution and pooling layer
    x = Conv3D(32, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # MobileNet blocks
    x = depthwise_separable_conv3d(x, 64)
    x = depthwise_separable_conv3d(x, 128, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 128)
    x = depthwise_separable_conv3d(x, 256, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 256)
    x = depthwise_separable_conv3d(x, 512, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 512)

    # Global pooling
    x = GlobalAveragePooling3D()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the MobileNet base model
mobilenet_base_model = create_mobilenet_base(input_shape=(11, 100, 100, 1))

# Assume X_train, X_test, y_train, y_test are predefined
# Example data initialization
# X_train, X_test, y_train, y_test = train_test_split(...)

# Extract features
X_train_features = mobilenet_base_model.predict(X_train)
X_test_features = mobilenet_base_model.predict(X_test)

# Train SVM classifier
svm_classifier = SVC(kernel='rbf', C=1.0)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_classifier", svm_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 33s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
Accuracy: 0.15
Precision: 0.07361111111111111
Recall: 0.15
F1 Score: 0.08818181818181818


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [42]:
from tensorflow.keras.layers import Input, Conv3D, GlobalAveragePooling3D, Dense, Dropout, Activation, BatchNormalization, ReLU
from tensorflow.keras.models import Model
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Define the depthwise separable convolution block
def depthwise_separable_conv3d(input_tensor, pointwise_filters, strides=(1, 1, 1)):
    channels = input_tensor.shape[-1]
    depthwise_conv = Conv3D(channels, kernel_size=(3, 3, 3), strides=strides, padding='same', groups=channels)(input_tensor)
    x = BatchNormalization()(depthwise_conv)
    x = ReLU()(x)
    
    pointwise_conv = Conv3D(pointwise_filters, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='same')(x)
    x = BatchNormalization()(pointwise_conv)
    x = ReLU()(x)
    return x

# Define the MobileNet model without the last layer
def create_mobilenet_base(input_shape):
    inputs = Input(shape=input_shape)

    # Initial convolution and pooling layer
    x = Conv3D(32, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # MobileNet blocks
    x = depthwise_separable_conv3d(x, 64)
    x = depthwise_separable_conv3d(x, 128, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 128)
    x = depthwise_separable_conv3d(x, 256, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 256)
    x = depthwise_separable_conv3d(x, 512, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 512)

    # Global pooling
    x = GlobalAveragePooling3D()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the MobileNet base model
mobilenet_base_model = create_mobilenet_base(input_shape=(11, 100, 100, 1))

# Assume X_train, X_test, y_train, y_test are predefined
# Example data initialization
# X_train, X_test, y_train, y_test = train_test_split(...)

# Extract features
X_train_features = mobilenet_base_model.predict(X_train)
X_test_features = mobilenet_base_model.predict(X_test)

# Train SVM classifier
svm_classifier = SVC(kernel='poly', degree=15)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_classifier", svm_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


4/4 ━━━━━━━━━━━━━━━━━━━━ 38s 9s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
Accuracy: 0.1
Precision: 0.05833333333333333
Recall: 0.1
F1 Score: 0.07333333333333333


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Mobile Net+KNN

In [43]:
from tensorflow.keras.layers import Input, Conv3D, GlobalAveragePooling3D, Dense, Dropout, Activation, BatchNormalization, ReLU
from tensorflow.keras.models import Model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Define the depthwise separable convolution block
def depthwise_separable_conv3d(input_tensor, pointwise_filters, strides=(1, 1, 1)):
    channels = input_tensor.shape[-1]
    depthwise_conv = Conv3D(channels, kernel_size=(3, 3, 3), strides=strides, padding='same', groups=channels)(input_tensor)
    x = BatchNormalization()(depthwise_conv)
    x = ReLU()(x)
    
    pointwise_conv = Conv3D(pointwise_filters, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='same')(x)
    x = BatchNormalization()(pointwise_conv)
    x = ReLU()(x)
    return x

# Define the MobileNet model without the last layer
def create_mobilenet_base(input_shape):
    inputs = Input(shape=input_shape)

    # Initial convolution and pooling layer
    x = Conv3D(32, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # MobileNet blocks
    x = depthwise_separable_conv3d(x, 64)
    x = depthwise_separable_conv3d(x, 128, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 128)
    x = depthwise_separable_conv3d(x, 256, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 256)
    x = depthwise_separable_conv3d(x, 512, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 512)

    # Global pooling
    x = GlobalAveragePooling3D()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the MobileNet base model
mobilenet_base_model = create_mobilenet_base(input_shape=(11, 100, 100, 1))

# Assume X_train, X_test, y_train, y_test are predefined
# Example data initialization
# X_train, X_test, y_train, y_test = train_test_split(...)

# Extract features
X_train_features = mobilenet_base_model.predict(X_train)
X_test_features = mobilenet_base_model.predict(X_test)

# Train k-NN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("knn_classifier", knn_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

4/4 ━━━━━━━━━━━━━━━━━━━━ 33s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
Accuracy: 0.1
Precision: 0.028333333333333332
Recall: 0.1
F1 Score: 0.04285714285714286


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [44]:
from tensorflow.keras.layers import Input, Conv3D, GlobalAveragePooling3D, Dense, Dropout, Activation, BatchNormalization, ReLU
from tensorflow.keras.models import Model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Define the depthwise separable convolution block
def depthwise_separable_conv3d(input_tensor, pointwise_filters, strides=(1, 1, 1)):
    channels = input_tensor.shape[-1]
    depthwise_conv = Conv3D(channels, kernel_size=(3, 3, 3), strides=strides, padding='same', groups=channels)(input_tensor)
    x = BatchNormalization()(depthwise_conv)
    x = ReLU()(x)
    
    pointwise_conv = Conv3D(pointwise_filters, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='same')(x)
    x = BatchNormalization()(pointwise_conv)
    x = ReLU()(x)
    return x

# Define the MobileNet model without the last layer
def create_mobilenet_base(input_shape):
    inputs = Input(shape=input_shape)

    # Initial convolution and pooling layer
    x = Conv3D(32, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # MobileNet blocks
    x = depthwise_separable_conv3d(x, 64)
    x = depthwise_separable_conv3d(x, 128, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 128)
    x = depthwise_separable_conv3d(x, 256, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 256)
    x = depthwise_separable_conv3d(x, 512, strides=(2, 2, 2))
    x = depthwise_separable_conv3d(x, 512)

    # Global pooling
    x = GlobalAveragePooling3D()(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Create the MobileNet base model
mobilenet_base_model = create_mobilenet_base(input_shape=(11, 100, 100, 1))

# Assume X_train, X_test, y_train, y_test are predefined
# Example data initialization
# X_train, X_test, y_train, y_test = train_test_split(...)

# Extract features
X_train_features = mobilenet_base_model.predict(X_train)
X_test_features = mobilenet_base_model.predict(X_test)

# Train k-NN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=15)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("knn_classifier", knn_classifier)
])
pipeline.fit(X_train_features, np.argmax(y_train, axis=1))

# Evaluate performance
y_pred = pipeline.predict(X_test_features)
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

4/4 ━━━━━━━━━━━━━━━━━━━━ 33s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
Accuracy: 0.1
Precision: 0.013333333333333332
Recall: 0.1
F1 Score: 0.023529411764705882


C:\Users\satvi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
